In [1]:
import pandas as pd
import pyodbc
import os
import useful_functions as use
from fuzzywuzzy import fuzz
import datetime

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
username = 'vigrose'
password = 'Ravenclaw~10946'
s = "DSN=PRDDM; UID={}; PWD={}".format(username, password)
AMADM = pyodbc.connect(s)

In [3]:
ppd = use.get_ppd()

C:/Users/vigrose/Data/PPD\ppd_data_20211127.csv


C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3249: DtypeWarning: Columns (13,14,20,21,49,56,57,72) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [4]:
x = "DSN=prdsso; UID={}; PWD={}".format(username, password)
SSO = pyodbc.connect(x)

org_query = \
        '''
        SELECT *
        FROM
        SSO.ORGANIZATION
        '''
org_addresses = pd.read_sql(con=SSO, sql=org_query)
org_addresses['ADVANTAGE_ID'] = org_addresses.ADVANTAGE_ID.fillna('0')
org_addresses = org_addresses[org_addresses.ADVANTAGE_ID!='']
org_addresses['CUSTOMER_NBR']=[float(x) for x in org_addresses.ADVANTAGE_ID]
org_addresses = org_addresses.sort_values('STATUS').drop_duplicates('ADVANTAGE_ID', keep='last')

In [5]:
order_query = \
    f""" 
    SELECT DISTINCT 
    D.FULL_DT,
    H.MED_EDU_NBR AS ME,
    H.PARTY_ID,
    O.ORDER_NBR,
    O.ORDER_PRODUCT_ID, 
    O.ORDER_PHYSICIAN_HIST_KEY,
    O.CUSTOMER_KEY
    FROM 
    AMADM.DIM_DATE D, AMADM.FACT_EPROFILE_ORDERS O, AMADM.DIM_PHYSICIAN_HIST H
    WHERE  
    D.DATE_KEY = O.ORDER_DT_KEY
    AND 
    H.PHYSICIAN_HIST_KEY = O.ORDER_PHYSICIAN_HIST_KEY
    """
customer_query = \
    """
    SELECT DISTINCT
    C.CUSTOMER_KEY, 
    C.CUSTOMER_NBR,
    C.CUSTOMER_ISELL_LOGIN,
    C.CUSTOMER_NAME,
    C.CUSTOMER_TYPE_DESC,
    C.CUSTOMER_TYPE,
    C.CUSTOMER_CATEGORY_DESC
    FROM 
    AMADM.dim_customer c  
    """    

In [6]:
orders = pd.read_sql(con=AMADM, sql=order_query)
customers = pd.read_sql(con=AMADM, sql=customer_query)
customers.CUSTOMER_KEY = customers.CUSTOMER_KEY.astype(str)
customers = customers.fillna('None')

In [7]:
ppd = ppd[['ME', 'POLO_MAILING_LINE_1', 'POLO_MAILING_LINE_2',
       'POLO_CITY', 'POLO_STATE', 'POLO_ZIP', 'TELEPHONE_NUMBER']]

In [8]:
orders = pd.merge(orders, ppd, on='ME')

In [9]:
customers['CUSTOMER_KEY'] = customers.CUSTOMER_KEY.astype(str)
customers.CUSTOMER_NBR = customers.CUSTOMER_NBR.astype(float)
named_customers = customers[customers.CUSTOMER_NAME != 'None']

In [10]:
results = pd.merge(orders, customers, on='CUSTOMER_KEY').drop_duplicates()
all_results = pd.merge(org_addresses, results, on='CUSTOMER_NBR')

In [11]:
# orders = orders[orders.ORDER_PRODUCT_ID.isin([4915514])]
# orders = orders[orders.ORDER_PRODUCT_ID.isin([4915513])]
these_orders = orders[orders.ORDER_PRODUCT_ID.isin([4915513,4915514])]

In [12]:
these_results = all_results[all_results.ORDER_PRODUCT_ID.isin([4915513,4915514])]

In [13]:
def clean(add_1):
    add_1 = str(add_1).strip().upper()
    return(add_1)

def is_a_match(thing_1, thing_2):
    thing_1 = clean(thing_1)
    thing_2 = clean(thing_2)
    if thing_1 == thing_2:
        match = True
    elif thing_1 in thing_2:
        match = True
    elif thing_2 in thing_1:
        match = True
    elif fuzz.ratio(thing_1, thing_2)>75:
        match = True
    else:
        match = False
    return(match)

def is_match(thing_1, thing_2):
    thing_1 = str(thing_1)
    thing_2 = str(thing_2)
    if thing_1 == thing_2:
        match = True
    elif thing_1 in thing_2:
        match = True
    elif thing_2 in thing_1:
        match = True
    else:
        match = False
    return(match)

In [14]:
zip_matches = []
add_matches = []
for row in these_results.itertuples():
    zip_match = False
    add_match =  False
    if row.STATE == row.POLO_STATE:
        if is_match(row.POLO_ZIP, row.ZIPCODE):
            zip_match = True
        if is_a_match(row.ADDRESS_LINE_1, row.POLO_MAILING_LINE_2):
            add_match = True
    zip_matches.append(zip_match)
    add_matches.append(add_match)

In [15]:
these_results['ZIP_MATCH'] = zip_matches
these_results['ADDRESS_MATCH'] = add_matches

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [32]:
these_results[(these_results.ZIP_MATCH==True)].groupby('CUSTOMER_CATEGORY_DESC').count()[['ORG_ID']]

,ORG_ID
CUSTOMER_CATEGORY_DESC,
Ambulatory Care,10090
Consulting,8
Credentialling Board,5
Credit Verification Org,339
Free Or Charitable Clinic,2
Government,58
Group,3185
Health Related,3692
Hospital,51949


In [24]:
these_results.columns

Index(['ORG_ID', 'ORG_NAME', 'PARTY_ID_x', 'STATUS', 'ADVANTAGE_ID',
       'ADDRESS_LINE_1', 'ADDRESS_LINE_2', 'CITY', 'STATE', 'ZIPCODE',
       'ORGANIZATION_TYPE', 'PHONE_NUMBER', 'CUSTOMER_NBR', 'FULL_DT', 'ME',
       'PARTY_ID_y', 'ORDER_NBR', 'ORDER_PRODUCT_ID',
       'ORDER_PHYSICIAN_HIST_KEY', 'CUSTOMER_KEY', 'POLO_MAILING_LINE_1',
       'POLO_MAILING_LINE_2', 'POLO_CITY', 'POLO_STATE', 'POLO_ZIP',
       'TELEPHONE_NUMBER', 'CUSTOMER_ISELL_LOGIN', 'CUSTOMER_NAME',
       'CUSTOMER_TYPE_DESC', 'CUSTOMER_TYPE', 'CUSTOMER_CATEGORY_DESC',
       'ZIP_MATCH', 'ADDRESS_MATCH', 'YEAR', 'MONTH'],
      dtype='object')

In [ ]:
these_results.FULL_DT = pd.to_datetime(these_results.FULL_DT)

In [18]:
these_results['YEAR'] = [x.year for x in these_results.FULL_DT]

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [19]:
these_results['MONTH'] = [x.month for x in these_results.FULL_DT]

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [20]:
these_results[these_results.YEAR==2021].nunique()['ME']

234874

In [ ]:
these_results.columns

In [21]:
a = these_results[these_results.ORDER_PRODUCT_ID==4915514].YEAR
b = these_results[these_results.ORDER_PRODUCT_ID==4915514].MONTH
pd.crosstab(b, [a], rownames=['MONTH'], colnames=['YEAR'])

YEAR,2014,2015,2016,2017,2018,2019,2020,2021
MONTH,,,,,,,,
1,0,5,7155,8046,8784,10081,9131,7306
2,0,4,7764,8693,6973,8235,8920,7835
3,0,5,7899,9083,8613,8055,9314,9334
4,0,19,8329,8742,8814,9412,7670,8541
5,0,338,6877,10340,7540,9461,7853,7038
6,0,11,7666,7823,6939,7799,8007,7875
7,0,5,6882,7675,7827,8918,7734,7529
8,1,14,8622,8251,8137,9009,7922,7165
9,0,6,7147,6609,7474,9532,7947,7268


In [35]:
these_results = these_results.fillna('None')

In [42]:
these_results[(these_results.CUSTOMER_CATEGORY_DESC.isin(cats))&(these_results.POLO_MAILING_LINE_2=='None')&(these_results.YEAR.isin([2020,2021]))].drop_duplicates('ME')

,ORG_ID,ORG_NAME,PARTY_ID_x,STATUS,ADVANTAGE_ID,ADDRESS_LINE_1,ADDRESS_LINE_2,CITY,STATE,ZIPCODE,...,TELEPHONE_NUMBER,CUSTOMER_ISELL_LOGIN,CUSTOMER_NAME,CUSTOMER_TYPE_DESC,CUSTOMER_TYPE,CUSTOMER_CATEGORY_DESC,ZIP_MATCH,ADDRESS_MATCH,YEAR,MONTH
1425,1650,Abrazo Community Health Network,None,0,000002083324,10020 North 25th Avenue,,Phoenix,AZ,85021,...,2.39624e+09,03HP74213,Abrazo Health Care,Eprofile Webstore Customer,Eprofile,Hospital,False,False,2020,2
1430,1650,Abrazo Community Health Network,None,0,000002083324,10020 North 25th Avenue,,Phoenix,AZ,85021,...,8.01224e+09,03HP74213,Abrazo Health Care,Eprofile Webstore Customer,Eprofile,Hospital,False,False,2020,1
1625,1650,Abrazo Community Health Network,None,0,000002083324,10020 North 25th Avenue,,Phoenix,AZ,85021,...,9.56351e+09,03HP74213,Abrazo Health Care,Eprofile Webstore Customer,Eprofile,Hospital,False,False,2020,1
1789,3826,"Harbor Hospital-Medstar Health, Inc.",None,0,000002086320,3001 S Hanover St,,Baltimore,MD,212251290,...,None,23HP62193,"Harbor Hospital-Medstar Health, Inc.",Eprofile Webstore Customer,Eprofile,Hospital,False,False,2020,1
1828,3826,"Harbor Hospital-Medstar Health, Inc.",None,0,000002086320,3001 S Hanover St,,Baltimore,MD,212251290,...,None,23HP62193,"Harbor Hospital-Medstar Health, Inc.",Eprofile Webstore Customer,Eprofile,Hospital,False,False,2020,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3372041,5758,Absentee Shawnee Tribal Health System,None,1,000002099221,15951 Little Axe Dr,,Norman,OK,730269088,...,None,38GR81011,"Li-Si-Wi-Nwi Health, Inc",Eprofile Webstore Customer,Eprofile,Group,False,False,2020,2
3372042,5758,Absentee Shawnee Tribal Health System,None,1,000002099221,15951 Little Axe Dr,,Norman,OK,730269088,...,5.80255e+09,38GR81011,"Li-Si-Wi-Nwi Health, Inc",Eprofile Webstore Customer,Eprofile,Group,False,False,2020,12
3372059,5758,Absentee Shawnee Tribal Health System,None,1,000002099221,15951 Little Axe Dr,,Norman,OK,730269088,...,4.05271e+09,38GR81011,"Li-Si-Wi-Nwi Health, Inc",Eprofile Webstore Customer,Eprofile,Group,False,False,2020,12
3372073,5758,Absentee Shawnee Tribal Health System,None,1,000002099221,15951 Little Axe Dr,,Norman,OK,730269088,...,None,38GR81011,"Li-Si-Wi-Nwi Health, Inc",Eprofile Webstore Customer,Eprofile,Group,False,False,2021,5


In [40]:
cats = ['Ambulatory Care',
                            'Hospital',
                            'Group',
                            'Government',
                            'Health Related',
                            'Government',
                            'Long Term Care',
                            'Managed Care']

In [57]:
these_results[(these_results.YEAR==2021)&(these_results.POLO_MAILING_LINE_2!='None')&(these_results.ADDRESS_MATCH==True)&(these_results.ZIP_MATCH==True)].groupby('MONTH').count()[['ORG_ID']]

,ORG_ID
MONTH,
1,412
2,290
3,450
4,501
5,381
6,470
7,437
8,383
9,286


In [44]:
len(ppd)

1307422

In [46]:
634875/1307422

0.485593021992899

In [ ]:
&(these_results.POLO_MAILING_LINE_2!='None')&(these_results.ADDRESS_MATCH==True)&(these_results.ZIP_MATCH==True)